In [1]:
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import numpy as np
import pickle
import pandas as pd
import os


tokenizer = AutoTokenizer.from_pretrained("vinai/PhoGPT-4B-Chat")
model = AutoModel.from_pretrained("vinai/PhoGPT-4B-Chat")

tokenizer_config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

In [2]:
def get_embedding(text):
    
    inputs = tokenizer(text, return_tensors="pt", truncation= True, padding= True, max_length = 512)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.detach().numpy().mean(1)
    return embedding

In [3]:
path_unsafe = "/kaggle/input/unsafety-data/merge.csv"
path_safe = "/kaggle/input/safety/alpaca_gpt4_data_vi_transformed.csv"

df_unsafe = pd.read_csv(path_unsafe)
df_safe = pd.read_csv(path_safe)

if os.path.exists('/kaggle/working/texts_labels.pkl'):
    with open('/kaggle/working/texts_labels.pkl', 'rb') as f:
        texts, labels = pickle.load(f)
    print("Loaded texts and labels from texts_labels.pkl")
else:
    texts = []
    labels = []
    print("File texts_labels.pkl does not exist. You may need to generate it first.")

File texts_labels.pkl does not exist. You may need to generate it first.


In [4]:
texts

[]

In [5]:
start = 0
end = 1000
index = 0
for _,row in df_unsafe.iterrows():
    if index < start:
        index += 1
        continue
    texts.append(get_embedding(row.iloc[0]))
    labels.append(0)
    print(index)
    index += 1
    if index == end:
        break


index = 0
for _,row in df_safe.iterrows():
    if index < start:
        index += 1
        continue
    texts.append(get_embedding(row.iloc[0]))
    labels.append(1)
    print(index)
    index += 1
    if index == end:
        break



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
with open('/kaggle/working/texts_labels.pkl', 'wb') as f:
    pickle.dump((texts, labels), f)

In [15]:
X = np.array(texts)
X = X.reshape(X.shape[0],-1)


y = np.array(labels)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(X_train.shape, X.shape, y.shape)
model_logistic = LogisticRegression(max_iter=1000)
model_logistic.fit(X_train, y_train)

y_pred = model_logistic.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

(1800, 3072) (2000, 3072) (2000,)
Accuracy: 0.995


In [8]:
import joblib

joblib.dump(model_logistic, '/kaggle/working/logistic_regression_model.pkl')
model_logistic = joblib.load('/kaggle/working/logistic_regression_model.pkl')

y_pred = model_logistic.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.995
